In [1]:
import os
os.chdir('/Users/richb/Desktop/COHP_toydata/jdftx_mod_test')

In [2]:
import numpy as np
outfile='water.out'


#Extract geometry from totalE.out:
iLine = 0
refLine = -10
R = np.zeros((3,3))
Rdone = False
ionPosStarted = False
ionNames = []
ionPos = []
for line in open(outfile):
	#Lattice vectors:
	if line.find('Initializing the Grid') >= 0 and (not Rdone):
		refLine = iLine
	rowNum = iLine - (refLine+2)
	if rowNum>=0 and rowNum<3:
		R[rowNum,:] = [ float(x) for x in line.split()[1:-1] ]
	if rowNum==3:
		refLine = -10
		Rdone = True
	#Coordinate system and ionic positions:
	if ionPosStarted:
		tokens = line.split()
		if len(tokens) and tokens[0] == 'ion':
			ionNames.append(tokens[1])
			ionPos.append([float(tokens[2]),float(tokens[3]),float(tokens[4])])
		else:
			break
	if line.find('# Ionic positions in') >= 0:
		coords = line.split()[4]
		ionPosStarted = True
	#Line counter:
	iLine += 1

In [19]:
def parse_complex_bandprojection(tokens):
	out = []
	for i in range(int(len(tokens)/2)):
		repart = tokens[2*i]
		impart = tokens[(2*i) + 1]
		num = complex(float(repart) + float(impart[1:]))
		out.append(num)
	return out

In [21]:
bandfile = 'water.bandProjections'

verbose = True
#Switch atoms to lattice coordinates:
ionPos = np.array(ionPos)
if coords != 'lattice':
	ionPos = np.dot(ionPos, np.linalg.inv(R.T)) #convert to lattice

#Read projection file:
for iLine,line in enumerate(open(bandfile)):
	tokens = line.split()
	if iLine==0:
		if verbose: print('line0 as tokens: ' + str(tokens))
		nStates = int(tokens[0])
		nBands = int(tokens[2])
		nProj = int(tokens[4])
		print('# of projections per state and band: ' + str(nProj))
		nSpecies = int(tokens[6])
		proj = np.zeros((nStates,nBands,nProj), dtype=complex)
		nOrbsPerAtom = [] #number of orbitals per atom
	elif iLine>=2:
		print('iLine=' + str(iLine))
		if iLine<nSpecies+2:
			print('passed first conditional')
			print('all tokens: ' + str(tokens))
			nAtoms = int(tokens[1])
			nOrbsPerAtom.extend( [int(tokens[2]),] * nAtoms )
			print('orbs per atom updated: ' + str(nOrbsPerAtom))
		else:
			print('didnt pass first conditional')
			iState = (iLine-(nSpecies+2)) // (nBands+1)
			print('iState: ' + str(iState))
			iBand = (iLine-(nSpecies+2)) - iState*(nBands+1) - 1 # This negative one makes the header line become iBand=-1
			print('iBand: ' + str(iBand))
			# if iBand < 0:
			# 	print('tokens for negative band (?)')
			# 	print(tokens)
			if iBand>=0 and iState<nStates:
				proj[iState,iBand]=np.array(parse_complex_bandprojection(tokens))
	print('---------------------------------')

line0 as tokens: ['8', 'states,', '4', 'bands,', '6', 'orbital-projections,', '2', 'species']
# of projections per state and band: 6
---------------------------------
---------------------------------
iLine=2
passed first conditional
all tokens: ['H', '2', '1', '0', '1']
orbs per atom updated: [1, 1]
---------------------------------
iLine=3
passed first conditional
all tokens: ['O', '1', '4', '2', '1', '1', '0']
orbs per atom updated: [1, 1, 4]
---------------------------------
iLine=4
didnt pass first conditional
iState: 0
iBand: -1
---------------------------------
iLine=5
didnt pass first conditional
iState: 0
iBand: 0
---------------------------------
iLine=6
didnt pass first conditional
iState: 0
iBand: 1
---------------------------------
iLine=7
didnt pass first conditional
iState: 0
iBand: 2
---------------------------------
iLine=8
didnt pass first conditional
iState: 0
iBand: 3
---------------------------------
iLine=9
didnt pass first conditional
iState: 1
iBand: -1
--------